<!-- 高亮文字 -->
<div style="background-color: #ffffcc; padding: 10px; border-radius: 5px; margin-bottom: 10px;">
  <strong>注意：</strong> 该仓库过于老旧，可能没法适配新的Hugging face等提供的接口
</div>

In [ ]:
import tensorflow as tf
import modeling

In [8]:
# 已经转换为 WordPiece 令牌 IDs
input_ids = tf.constant([[31, 51, 99], [15, 5, 0]])  # (B:2, seq_len: 3)
print(input_ids)
input_mask = tf.constant([[1, 1, 1], [1, 1, 0]])  # mask: 掩码1秒时有含义，0表示pad
token_type_ids = tf.constant([[0, 0, 1], [0, 2, 0]])

config = modeling.BertConfig(vocab_size=32000, hidden_size=512,
  num_hidden_layers=8, num_attention_heads=8, intermediate_size=1024)

model = modeling.BertModel(config=config, is_training=True,
  input_ids=input_ids, input_mask=input_mask, token_type_ids=token_type_ids)


num_labels = 10  # 关键：根据你的任务修改（例如二分类=2、多分类=10、NER=12等）
initializer_range = config.initializer_range  # 复用 BERT 配置的初始化标准差（默认 0.02）
label_embeddings = tf.Variable(
    initial_value=tf.random.truncated_normal(
        shape=[config.hidden_size, num_labels],  # 512 (hidden_size) × 10 (num_labels)
        stddev=initializer_range,  # 遵循 BERT 初始化逻辑，避免权重过大
        dtype=tf.float32
    ),
    name="label_embeddings",  # 变量名称，便于后续调试和保存
    trainable=True  # 训练过程中更新嵌入矩阵（关键：若固定标签嵌入可设为 False）
)
pooled_output = model.get_pooled_output()
# 含义：
# - BERT 输出有两个核心部分：
#   a. sequence_output(T1...TN)：序列输出 → 形状 (2, 3, 512) → 每个令牌的特征向量（用于序列标注任务，如 NER）
#   b. pooled_output(C)：池化输出 → 形状 (2, 512) → 整个文本的全局特征向量（用于分类任务）
# - 池化逻辑：通常取第一个令牌 [CLS] 的特征向量，再经过全连接层和激活函数得到
# - 用途：作为整个文本的“语义摘要”，用于文本分类、情感分析等任务
logits = tf.matmul(pooled_output, label_embeddings)  # 通过线性分类器

# 验证输出形状（可选）
print("Pooled output shape:", pooled_output.shape)  # 输出 (2, 512) → (batch_size=2, hidden_size=512)
print("Label embeddings shape:", label_embeddings.shape)  # 输出 (512, 10) → (hidden_size=512, num_labels=10)
print("Logits shape:", logits.shape)  # 输出 (2, 10) → (batch_size=2, num_labels=10)

tf.Tensor(
[[31 51 99]
 [15  5  0]], shape=(2, 3), dtype=int32)
Pooled output shape: (2, 512)
Label embeddings shape: (512, 10)
Logits shape: (2, 10)
